In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime as dt
from dateutil.parser import *
import random
import time
import json
from numpy import argmax
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score,classification_report, roc_auc_score,accuracy_score, \
precision_score, f1_score, recall_score, roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import re
import seaborn as sns
from imblearn.over_sampling import SMOTE 

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

%config InlineBackend.figure_format = 'retina'
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12)

In [2]:
df_regression = pd.read_csv('../data/merged_train_final.csv')

In [3]:
test_df_regression = pd.read_csv('../data/merged_test_final.csv')

In [4]:
merged_train_multi = pd.read_csv('../data/merged_train_multi.csv')

In [5]:
merged_test_multi = pd.read_csv('../data/merged_test_multi.csv')

In [6]:
x = df_regression.drop(['wnvpresent'],axis=1)
y = df_regression['wnvpresent']

#x = merged_train_multi.drop(['wnvpresent'],axis=1)
#y = merged_train_multi['wnvpresent']

display(y.value_counts(normalize=True))

x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [7]:
# Instantiate models
models = {'lr': LogisticRegression(max_iter=5_000, random_state=42, solver='saga'),
          'rf': RandomForestClassifier(random_state=42),
          'gb': GradientBoostingClassifier(random_state=42),
          'et': ExtraTreesClassifier(random_state=42),
          'ada': AdaBoostClassifier(random_state=42),
          'svm': SVC(random_state=42, probability=True),
          'xg': XGBClassifier(random_state=42)
}

In [8]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1)
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [9]:
# create list to store results
model_eval =[]

In [10]:
lr_base = get_model_scores('lr_base', 'lr')

METRICS-->


{'model_name': 'lr_base',
 'model': 'lr',
 'train_score': 0.8057329462989841,
 'test_score': 0.7541254125412541,
 'recall': 0.5887096774193549,
 'specificity': 0.7630434782608696,
 'precision': 0.11812297734627832,
 'train_auc': 0.8751527212826059,
 'test_auc': 0.7461448106591865,
 'is_tuned': False}

True Negatives: 1755
False Positives: 545
False Negatives: 51
True Positives: 73


In [11]:
svm_base = get_model_scores('svm_base', 'svm')

METRICS-->


{'model_name': 'svm_base',
 'model': 'svm',
 'train_score': 0.9130624092888244,
 'test_score': 0.8407590759075908,
 'recall': 0.4274193548387097,
 'specificity': 0.8630434782608696,
 'precision': 0.14402173913043478,
 'train_auc': 0.9712810577160058,
 'test_auc': 0.7514708976157083,
 'is_tuned': False}

True Negatives: 1985
False Positives: 315
False Negatives: 71
True Positives: 53


In [12]:
xg_base = get_model_scores('xg_base', 'xg')

[16:03:14] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


METRICS-->


{'model_name': 'xg_base',
 'model': 'xg',
 'train_score': 0.9776487663280116,
 'test_score': 0.9426567656765676,
 'recall': 0.08064516129032258,
 'specificity': 0.9891304347826086,
 'precision': 0.2857142857142857,
 'train_auc': 0.9971802490304831,
 'test_auc': 0.8572072230014025,
 'is_tuned': False}

True Negatives: 2275
False Positives: 25
False Negatives: 114
True Positives: 10


In [13]:
et_base = get_model_scores('et_base', 'et')

METRICS-->


{'model_name': 'et_base',
 'model': 'et',
 'train_score': 0.9918722786647315,
 'test_score': 0.941006600660066,
 'recall': 0.07258064516129033,
 'specificity': 0.9878260869565217,
 'precision': 0.24324324324324326,
 'train_auc': 0.9998393056131918,
 'test_auc': 0.6866374474053296,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 115
True Positives: 9


In [14]:
model_eval

[{'model_name': 'lr_base',
  'model': 'lr',
  'train_score': 0.8057329462989841,
  'test_score': 0.7541254125412541,
  'recall': 0.5887096774193549,
  'specificity': 0.7630434782608696,
  'precision': 0.11812297734627832,
  'train_auc': 0.8751527212826059,
  'test_auc': 0.7461448106591865,
  'is_tuned': False},
 {'model_name': 'svm_base',
  'model': 'svm',
  'train_score': 0.9130624092888244,
  'test_score': 0.8407590759075908,
  'recall': 0.4274193548387097,
  'specificity': 0.8630434782608696,
  'precision': 0.14402173913043478,
  'train_auc': 0.9712810577160058,
  'test_auc': 0.7514708976157083,
  'is_tuned': False},
 {'model_name': 'xg_base',
  'model': 'xg',
  'train_score': 0.9776487663280116,
  'test_score': 0.9426567656765676,
  'recall': 0.08064516129032258,
  'specificity': 0.9891304347826086,
  'precision': 0.2857142857142857,
  'train_auc': 0.9971802490304831,
  'test_auc': 0.8572072230014025,
  'is_tuned': False},
 {'model_name': 'et_base',
  'model': 'et',
  'train_score'

In [15]:
#x = df_regression.drop(['wnvpresent'],axis=1)
#y = df_regression['wnvpresent']

x = merged_train_multi.drop(['wnvpresent'],axis=1)
y = merged_train_multi['wnvpresent']

display(y.value_counts(normalize=True))

x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [36]:
lr_params = {
    # Trying different types of regularization
    'lr__penalty':['l2','l1', 'elasticnet'],

     # Trying different alphas of: 1, 0.1, 0.05  (C = 1/alpha)
    'lr__C':[1, 10, 20],

}

svc_params = {
    'svc__C':[10, 30],
    'svc__gamma':[0.01, 0.1], 
    'svc__kernel':['rbf', 'sigmoid'],
}

In [16]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1)
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval_2.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [17]:
model_eval_2 = []

In [18]:
lr_base = get_model_scores('lr_base', 'lr')

METRICS-->


{'model_name': 'lr_base',
 'model': 'lr',
 'train_score': 0.8259796806966618,
 'test_score': 0.775990099009901,
 'recall': 0.6048387096774194,
 'specificity': 0.7852173913043479,
 'precision': 0.13181019332161686,
 'train_auc': 0.8874044228083443,
 'test_auc': 0.7729926367461429,
 'is_tuned': False}

True Negatives: 1806
False Positives: 494
False Negatives: 49
True Positives: 75


In [19]:
svm_base = get_model_scores('svm_base', 'svm')

METRICS-->


{'model_name': 'svm_base',
 'model': 'svm',
 'train_score': 0.9298984034833091,
 'test_score': 0.8535478547854786,
 'recall': 0.45161290322580644,
 'specificity': 0.8752173913043478,
 'precision': 0.16326530612244897,
 'train_auc': 0.9783968162352202,
 'test_auc': 0.777312412342216,
 'is_tuned': False}

True Negatives: 2013
False Positives: 287
False Negatives: 68
True Positives: 56


In [20]:
xg_base = get_model_scores('xg_base', 'xg')

[16:05:26] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/user_1/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


METRICS-->


{'model_name': 'xg_base',
 'model': 'xg',
 'train_score': 0.978011611030479,
 'test_score': 0.9422442244224423,
 'recall': 0.0967741935483871,
 'specificity': 0.9878260869565217,
 'precision': 0.3,
 'train_auc': 0.9974362836276467,
 'test_auc': 0.8528962131837308,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 112
True Positives: 12


In [21]:
et_base = get_model_scores('et_base', 'et')

METRICS-->


{'model_name': 'et_base',
 'model': 'et',
 'train_score': 0.9918722786647315,
 'test_score': 0.9426567656765676,
 'recall': 0.06451612903225806,
 'specificity': 0.99,
 'precision': 0.25806451612903225,
 'train_auc': 0.9998393056131918,
 'test_auc': 0.70117110799439,
 'is_tuned': False}

True Negatives: 2277
False Positives: 23
False Negatives: 116
True Positives: 8


In [23]:
model_eval_2

[{'model_name': 'lr_base',
  'model': 'lr',
  'train_score': 0.8259796806966618,
  'test_score': 0.775990099009901,
  'recall': 0.6048387096774194,
  'specificity': 0.7852173913043479,
  'precision': 0.13181019332161686,
  'train_auc': 0.8874044228083443,
  'test_auc': 0.7729926367461429,
  'is_tuned': False},
 {'model_name': 'svm_base',
  'model': 'svm',
  'train_score': 0.9298984034833091,
  'test_score': 0.8535478547854786,
  'recall': 0.45161290322580644,
  'specificity': 0.8752173913043478,
  'precision': 0.16326530612244897,
  'train_auc': 0.9783968162352202,
  'test_auc': 0.777312412342216,
  'is_tuned': False},
 {'model_name': 'xg_base',
  'model': 'xg',
  'train_score': 0.978011611030479,
  'test_score': 0.9422442244224423,
  'recall': 0.0967741935483871,
  'specificity': 0.9878260869565217,
  'precision': 0.3,
  'train_auc': 0.9974362836276467,
  'test_auc': 0.8528962131837308,
  'is_tuned': False},
 {'model_name': 'et_base',
  'model': 'et',
  'train_score': 0.99187227866473

In [28]:
eval_df = pd.concat([pd.DataFrame(model_eval),pd.DataFrame(model_eval_2)])
eval_df

,model_name,model,train_score,test_score,recall,specificity,precision,train_auc,test_auc,is_tuned
0,lr_base,lr,0.805733,0.754125,0.588710,0.763043,0.118123,0.875153,0.746145,False
1,svm_base,svm,0.913062,0.840759,0.427419,0.863043,0.144022,0.971281,0.751471,False
2,xg_base,xg,0.977649,0.942657,0.080645,0.989130,0.285714,0.997180,0.857207,False
3,et_base,et,0.991872,0.941007,0.072581,0.987826,0.243243,0.999839,0.686637,False
0,lr_base,lr,0.825980,0.775990,0.604839,0.785217,0.131810,0.887404,0.772993,False
1,svm_base,svm,0.929898,0.853548,0.451613,0.875217,0.163265,0.978397,0.777312,False
2,xg_base,xg,0.978012,0.942244,0.096774,0.987826,0.300000,0.997436,0.852896,False
3,et_base,et,0.991872,0.942657,0.064516,0.990000,0.258065,0.999839,0.701171,False


In [31]:
overtrain_s = eval_df['train_auc'] - eval_df['test_auc']

# remove multi with vif
overtrain_s[:4]

0    0.129008
1    0.219810
2    0.139973
3    0.313202
dtype: float64

In [35]:
overtrain_s[:4].mean()

0.20049823874016492

In [33]:
# ignore multi, no vif
overtrain_s[4:]

0    0.114412
1    0.201084
2    0.144540
3    0.298668
dtype: float64

In [34]:
overtrain_s[4:].mean()

0.18967611450448082